In [9]:
'''Changing the txt file to csv file'''

# import csv

# txt_file = r"/Users/liyatilahun/Downloads/booksummaries/booksummaries.txt"
# csv_file = r"book_detail.csv"

# with open(txt_file, "r") as in_text:
#     in_reader = csv.reader(in_text, delimiter = '\t')
#     with open(csv_file, "w") as out_csv:
#         out_writer = csv.writer(out_csv)
#         for row in in_reader:
#             out_writer.writerow(row)

In [44]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer #histogram(array) representation of words
from sklearn.metrics.pairwise import cosine_similarity #to calculate the cosine_similarity
from sklearn import datasets
from scipy import sparse
import json

#data from goodbooks-10k dataset
df = pd.read_csv('book_detail.csv')
df.drop(df.columns[[-1,-6,-7]], axis=1, inplace=True) #delete these column

def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)
        
display_all(df.head())
# print(df.keys())





,title,author,year,genre
0,Animal Farm,George Orwell,1945-08-17,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"":..."
1,A Clockwork Orange,Anthony Burgess,1962,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N..."
2,The Plague,Albert Camus,1947,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fi..."
3,An Enquiry Concerning Human Understanding,David Hume,NaN,NaN
4,A Fire Upon the Deep,Vernor Vinge,NaN,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90..."


In [45]:
df['genre']
df.shape

(16559, 4)

In [3]:
df['year']

0        1945-08-17
1              1962
2              1947
3               NaN
4               NaN
            ...    
16554    2012-09-25
16555    2000-06-01
16556    2010-11-16
16557    2012-10-02
16558          1846
Name: year, Length: 16559, dtype: object

In [46]:
df.drop(df[df['genre']==''].index, inplace=True)

In [47]:
df.shape

(16559, 4)

In [24]:
df[df['genre']=='']

,title,author,year,genre


In [48]:
json.loads(df['genre'][0]).values() #loading values from the json column for the first row

dict_values(['Roman à clef', 'Satire', "Children's literature", 'Speculative fiction', 'Fiction'])

In [50]:
df = df[df['genre'].notna()] #drop the NaN rows in column 'genre'
# get the values from the genre column json object and append to a list
genres = []
for i in df['genre']:
    genres.append(list(json.loads(i).values()))
    
# assign it to a new column
df['genre_new'] = genres


In [53]:
df['genre']

0        {"/m/016lj8": "Roman \u00e0 clef", "/m/06nbt":...
1        {"/m/06n90": "Science Fiction", "/m/0l67h": "N...
2        {"/m/02m4t": "Existentialism", "/m/02xlf": "Fi...
4        {"/m/03lrw": "Hard science fiction", "/m/06n90...
5        {"/m/098tmk": "War novel", "/m/016lj8": "Roman...
                               ...                        
16549                      {"/m/06n90": "Science Fiction"}
16551    {"/m/01jfsb": "Thriller", "/m/02xlf": "Fiction...
16555     {"/m/01jfsb": "Thriller", "/m/02xlf": "Fiction"}
16556                         {"/m/0xdf": "Autobiography"}
16558    {"/m/02ql9": "Epistolary novel", "/m/014dfn": ...
Name: genre, Length: 12841, dtype: object

In [54]:
df.shape

(12841, 5)

In [55]:
df['genre_new']

0        [Roman à clef, Satire, Children's literature, ...
1        [Science Fiction, Novella, Speculative fiction...
2        [Existentialism, Fiction, Absurdist fiction, N...
4        [Hard science fiction, Science Fiction, Specul...
5                                [War novel, Roman à clef]
                               ...                        
16549                                    [Science Fiction]
16551                        [Thriller, Fiction, Suspense]
16555                                  [Thriller, Fiction]
16556                                      [Autobiography]
16558              [Epistolary novel, Speculative fiction]
Name: genre_new, Length: 12841, dtype: object

In [60]:
#drop the genre column
df.drop(df.columns[[-2]], axis=1, inplace=True)
df

,title,author,year,genre,genre_new
0,Animal Farm,George Orwell,1945-08-17,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"":...","[Roman à clef, Satire, Children's literature, ..."
1,A Clockwork Orange,Anthony Burgess,1962,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N...","[Science Fiction, Novella, Speculative fiction..."
2,The Plague,Albert Camus,1947,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fi...","[Existentialism, Fiction, Absurdist fiction, N..."
4,A Fire Upon the Deep,Vernor Vinge,NaN,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90...","[Hard science fiction, Science Fiction, Specul..."
5,All Quiet on the Western Front,Erich Maria Remarque,1929-01-29,"{""/m/098tmk"": ""War novel"", ""/m/016lj8"": ""Roman...","[War novel, Roman à clef]"
...,...,...,...,...,...
16542,The Bourne Imperative,Robert Ludlum,2012-06-05,"{""/m/01jfsb"": ""Thriller"", ""/m/06wkf"": ""Spy fic...","[Thriller, Spy fiction]"
16544,Heaven,V. C. Andrews,1985-10-15,"{""/m/039vk"": ""Gothic fiction"", ""/m/03mfnf"": ""Y...","[Gothic fiction, Young adult literature, Fiction]"
16545,Dark Angel,V. C. Andrews,1986-11,"{""/m/039vk"": ""Gothic fiction"", ""/m/03mfnf"": ""Y...","[Gothic fiction, Young adult literature]"
16546,Gates of Paradise,V. C. Andrews,1989-06,"{""/m/039vk"": ""Gothic fiction"", ""/m/03mfnf"": ""Y...","[Gothic fiction, Young adult literature]"
